In [ ]:
import os
import shutil
import time
import numpy as np
import geojson
import geopandas as gpd
import shapely.wkt
from pathlib import Path
from sentinel2download.downloader import Sentinel2Downloader
from sentinel2download.overlap import Sentinel2Overlap

from code.download.utils import get_tiles, check_nodata, get_min_clouds
from code.download.load_tiles import load_images
from code.index_research import calculate_ndvi
from code.plant_stress import PlantStress
from code.utils import dump_no_data_geosjon, stitch_tiles

import warnings
warnings.filterwarnings('ignore')
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [ ]:
REQUEST_ID=os.getenv('REQUEST_ID')
START_DATE=os.getenv('START_DATE')
END_DATE=os.getenv('END_DATE')
AOI=os.getenv('AOI')
SENTINEL2_GOOGLE_API_KEY=os.getenv('SENTINEL2_GOOGLE_API_KEY')
SATELLITE_CACHE_FOLDER=os.getenv('SENTINEL2_CACHE')
OUTPUT_FOLDER=os.getenv('OUTPUT_FOLDER')

In [ ]:
default_crs = 'EPSG:4326'

polygon = shapely.wkt.loads(AOI)
aoi_filename = f"{time.time()}_aoi.geojson"
gpd.GeoDataFrame(gpd.GeoSeries([polygon]), columns=["geometry"]).to_file(aoi_filename, driver="GeoJSON")

In [ ]:
BASE = os.getcwd()
BANDS = {'B04','B08'}
CONSTRAINTS = {'NODATA_PIXEL_PERCENTAGE': 15.0, 'CLOUDY_PIXEL_PERCENTAGE': 10.0, }
PRODUCT_TYPE = 'L2A'
NAME = 'Field anomalies'
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

In [ ]:
b04_tiles, b08_tiles, tci_tiles = [], [], []
s2overlap = Sentinel2Overlap(aoi_path=aoi_filename)
overlap_tiles = s2overlap.overlap_with_geometry()

In [ ]:
loadings = load_images(SENTINEL2_GOOGLE_API_KEY, overlap_tiles.Name.values, START_DATE, END_DATE, SATELLITE_CACHE_FOLDER, BANDS, CONSTRAINTS, PRODUCT_TYPE)
checked = check_nodata(loadings, PRODUCT_TYPE)
needed_num_tiles = len(checked.keys())

while len(b04_tiles) < needed_num_tiles and len(b08_tiles) < needed_num_tiles:
    if CONSTRAINTS['NODATA_PIXEL_PERCENTAGE'] > 70:
        metadata = dict(START_DATE=START_DATE, END_DATE=END_DATE, REQUEST_ID=REQUEST_ID, NAME=NAME)
        geojson_path = os.path.join(OUTPUT_FOLDER, f"{START_DATE}_{END_DATE}_no_data.geojson")
        dump_no_data_geosjon(polygon, geojson_path, metadata)
        raise ValueError("Images not loaded for given AOI. Change dates, constraints")

    for i, tile in overlap_tiles.iterrows():
        if len(checked[tile.Name]) < 1:
            continue
                
        tile_folder = Path(sorted(checked[tile.Name])[0])
        tile_files = list(tile_folder.rglob('*.jp2'))
        b04_tiles.append([x for x in tile_files if '_B04_' in str(x)][0])
        b08_tiles.append([x for x in tile_files if '_B08_' in str(x)][0])
        print(f'filtered: {tile_folder}')

    CONSTRAINTS['NODATA_PIXEL_PERCENTAGE'] += 5
    CONSTRAINTS['CLOUDY_PIXEL_PERCENTAGE'] += 5
    loadings = load_images(SENTINEL2_GOOGLE_API_KEY, overlap_tiles.Name.values, START_DATE, END_DATE, SATELLITE_CACHE_FOLDER, BANDS, CONSTRAINTS, PRODUCT_TYPE)
    checked = check_nodata(loadings, PRODUCT_TYPE)

In [ ]:
if len(b04_tiles) > 1:
    b04_tile = stitch_tiles(b04_tiles, str(b04_tiles[0]).replace('.jp2', '_merged.tif'))
    b08_tile = stitch_tiles(b08_tiles, str(b08_tiles[0]).replace('.jp2', '_merged.tif'))
else:
    b04_tile = str(b04_tiles[0])
    b08_tile = str(b08_tiles[0])

In [ ]:
ndvi_path = os.path.join(BASE, f'{START_DATE}_{END_DATE}_ndvi.tif')
if not os.path.exists(ndvi_path):
    calculate_ndvi(b04_tile, b08_tile, out_path=ndvi_path, nodata=np.nan)

In [ ]:
min_ndvi = 0.3
z_score = 5
z_score_anom = 1
colors = {"Normal Growth": (0, 0, 0), "Anomaly": (182, 10, 28)}

field = gpd.read_file(aoi_filename)

In [ ]:
ps = PlantStress(min_ndvi=min_ndvi, noise_z_score=z_score, anomaly_z_score=z_score_anom)

In [ ]:
raster_path = os.path.join(OUTPUT_FOLDER, f'{START_DATE}_{END_DATE}_field.tif')
ps.segment_field(NAME, field, ndvi_path, raster_path, START_DATE, END_DATE, REQUEST_ID)